# Integration Test: CandleDataAPI

CandleDataAPI를 통한 데이터 수집 및 로드 테스트

In [1]:
import sys

from pathlib import Path
from loguru import logger

# 기존 핸들러 제거 후 INFO 레벨로 재설정
logger.remove()
logger.add(sys.stderr, level="INFO")

# src 경로 추가
sys.path.insert(0, str(Path.cwd() / 'src'))

from candle_data_manager.API.CandleDataAPI.CandleDataAPI import CandleDataAPI

print("Import 완료")

Import 완료


## 1. CandleDataAPI 초기화

In [2]:
# CandleDataAPI 초기화 (내부에서 자동 조립)
api = CandleDataAPI()

print("CandleDataAPI 초기화 완료")

2026-01-21 18:00:34.408 | INFO     | candle_data_manager.Core.ConnectionManager.ConnectionManager:__init__:36 - ConnectionManager 초기화 완료: pool_size=5, max_overflow=10
2026-01-21 18:00:34.414 | INFO     | candle_data_manager.Service.SymbolService.SymbolService:__init__:14 - SymbolService 초기화 완료
2026-01-21 18:00:34.414 | INFO     | candle_data_manager.Plugin.SymbolPreparationPlugin.SymbolPreparationPlugin:__init__:14 - SymbolPreparationPlugin 초기화 완료
2026-01-21 18:00:34.414 | INFO     | candle_data_manager.Plugin.UpdateOrchestrationPlugin.UpdateOrchestrationPlugin:__init__:31 - UpdateOrchestrationPlugin 초기화 완료 (buffer_size=50000)
2026-01-21 18:00:34.416 | INFO     | candle_data_manager.API.CandleDataAPI.CandleDataAPI:__init__:53 - CandleDataAPI 초기화 완료


CandleDataAPI 초기화 완료


## 2. Active Update 테스트 (Upbit KRW 마켓)

In [3]:
# Upbit KRW 마켓 Active Update (1d 타임프레임만)
# 주의: 전체 마켓 업데이트는 시간이 오래 걸릴 수 있음

result = api.active_update(
    archetype="CRYPTO",
    exchange="BINANCe",
    tradetype="FUTURES",
    base="BTC",
    quote="USDT",
    timeframe="1m"
)

print(f"Active Update 결과:")
print(f"  - 성공: {len(result.success_symbols)}개")
print(f"  - 실패: {len(result.failed_symbols)}개")
print(f"  - 총 rows: {result.total_rows}")

2026-01-21 18:02:22.832 | WARNING  | candle_data_manager.Core.Utils.Throttler.SyncThrottler:wait:78 - [BinanceFutures] Rate limit 대기: 0.008초
2026-01-21 18:02:22.912 | WARNING  | candle_data_manager.Core.Utils.Throttler.SyncThrottler:wait:78 - [BinanceFutures] Rate limit 대기: 0.025초
2026-01-21 18:02:23.003 | WARNING  | candle_data_manager.Core.Utils.Throttler.SyncThrottler:wait:78 - [BinanceFutures] Rate limit 대기: 0.013초
2026-01-21 18:02:23.081 | WARNING  | candle_data_manager.Core.Utils.Throttler.SyncThrottler:wait:78 - [BinanceFutures] Rate limit 대기: 0.170초
2026-01-21 18:02:23.371 | WARNING  | candle_data_manager.Core.Utils.Throttler.SyncThrottler:wait:78 - [BinanceFutures] Rate limit 대기: 0.048초
2026-01-21 18:02:23.551 | WARNING  | candle_data_manager.Core.Utils.Throttler.SyncThrottler:wait:78 - [BinanceFutures] Rate limit 대기: 0.011초
2026-01-21 18:02:23.654 | WARNING  | candle_data_manager.Core.Utils.Throttler.SyncThrottler:wait:78 - [BinanceFutures] Rate limit 대기: 0.038초
2026-01-21 18

Active Update 결과:
  - 성공: 1개
  - 실패: 0개
  - 총 rows: 3350343


## 3. Load 테스트

In [ ]:
# Upbit KRW BTC 1d 로드
from datetime import datetime, timedelta

end_ts = int(datetime.now().timestamp())
start_ts = int((datetime.now() - timedelta(days=30)).timestamp())

markets = api.load(
    archetype="CRYPTO",
    exchange="UPBIT",
    tradetype="SPOT",
    base="BTC",
    quote="KRW",
    timeframe="1d",
    start_at=start_ts,
    end_at=end_ts
)

print(f"로드된 Market 수: {len(markets)}")
if markets:
    market = markets[0]
    print(f"Symbol: {market.symbol.to_string()}")
    print(f"데이터 rows: {len(market.candles)}")
    print(market.candles.tail())

## 4. 정리

In [ ]:
api.close()
print("테스트 완료")